In [1]:
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
# from torch.linalg import svd
from torch.linalg import svd

In [2]:
model_name = "meta-llama/Llama-3.2-1B" # "meta-llama/Meta-Llama-3.1-8B"  # "meta-llama/Meta-Llama-3.1-8B" , "gpt2", "meta-llama/Llama-2-7b-hf" # Example: using GPT-2
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# %%
# Loading embeddings:
# embedding = model.model.embed_tokens.weight  # Embedding layer weights
head = model.lm_head.weight.detach()  # Output head weights
head_norm = head / torch.norm(head)

In [3]:
embeddings = head

In [4]:
# def find_self_embeds(embedding):
start = 0
end = len(embeddings)
all_indices = np.array(range(end))
batch_size = 10000
batches = [
    range(i, min(i + batch_size, end)) for i in range(start, end, batch_size)
]
eqs = []
embeddings = embeddings.cuda()

for batch in tqdm(batches):
    indices = np.array(list(batch))
    X = embeddings[batch]
    w = np.array(torch.argmax(X @ embeddings.T, dim=-1).cpu())
    eq = indices == w
    eqs.extend(eq)
eqs = np.array(eqs)
print(f"failed matching = {sum(~eqs)}")
fail_indices = all_indices[~eqs]

  0%|          | 0/13 [00:00<?, ?it/s]C:\Users\Timur\AppData\Local\Temp\ipykernel_5168\2878507353.py:15: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  w = np.array(torch.argmax(X @ embeddings.T, dim=-1).cpu())
100%|██████████| 13/13 [00:13<00:00,  1.07s/it]

failed matching = 615


In [5]:
shadow_ratios = []
for ind in fail_indices:
    max_sims = torch.sort(embeddings[ind]@embeddings.T, descending=True)[0][:3]
    max_sim = max_sims[0]
    true_sim = embeddings[ind]@embeddings[ind]
    true_ratio = true_sim / max_sim
    max_ratio = max_sims[1] / max_sim
    max_ratio_2 = max_sims[2] / max_sim
    shadow_ratios.append((ind, true_ratio.item(), max_ratio.item(), max_ratio_2.item()))
shadow_ratios_sorted = sorted(shadow_ratios, key=lambda x: x[1], reverse = True);

In [6]:
n = 10
embeddings_cut = embeddings[:10000]
embeddings_others = torch.cat((embeddings_cut[:n], embeddings_cut[n+1:]), dim=0).detach().cpu()
self_emb = embeddings_cut[n].detach().cpu()

In [7]:
A = self_emb - embeddings_others
torch.all(A@self_emb>0)

tensor(True)

In [8]:
from scipy.optimize import linprog

In [ ]:
n, d = A.shape  # A is (n, d), x is (n,)
epsilon=1e-6

c = np.zeros(d)  # No objective function, just feasibility
bounds = [(None, None)] * d  # x can take any value

res = linprog(c, A_ub=-A, b_ub=-epsilon * np.ones(n), bounds=bounds, method='highs')

In [22]:
# from numpy.linalg import svd
# from scipy.linalg import svd
from torch.linalg import svd

In [67]:
# U, S, Vt = svd(A.T, full_matrices=True)
U, S, Vt = svd(A.T)

In [68]:
# Null space basis vectors correspond to zero singular values
tol = 1e-10  # Tolerance to detect numerical zeros
null_space = Vt[S < tol] # Extract null space basis

IndexError: The shape of the mask [2048] at index 0 does not match the shape of the indexed tensor [9999, 9999] at index 0

In [70]:
torch.linalg.matrix_rank(A.T)

tensor(2048)

In [73]:
import numpy as np
from scipy.optimize import linprog

In [86]:
d, n = A.T.shape
epsilon=1e-6

c = np.zeros(n)
bounds = [(epsilon, None)] * n  # Enforce strict positivity (x_i > epsilon)
res = linprog(c, A_eq=A.T, b_eq=np.zeros(d), bounds=bounds, method='highs')

In [78]:
qq = res

In [83]:
res

       message: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is Infeasible)
       success: False
        status: 2
           fun: None
             x: None
           nit: 69
         lower:  residual: None
                marginals: None
         upper:  residual: None
                marginals: None
         eqlin:  residual: None
                marginals: None
       ineqlin:  residual: None
                marginals: None